In [1]:
import torch
import random
import numpy as np
import torch.backends.cudnn as cudnn
import re
import pandas as pd
from konlpy.tag import Komoran
from dateutil import parser
from datetime import datetime, timedelta
import calendar

torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.cuda.manual_seed_all(42)
np.random.seed(42)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(42)

In [2]:
# 저장한 모델 불러오기
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

from transformers import AutoModel, AutoTokenizer, BertTokenizer
MODEL_NAME = "beomi/KcELECTRA-base-v2022"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

tag2id = {'POH_B': 0, 'CVL_B': 1, 'FLD_B': 2, 'FLD_I': 3, 'EVT_I': 4,'AFW_B': 5,'CVL_I': 6,'NUM_B': 7,'MAT_I': 8, 'TIM_I': 9,'ANM_I': 10,'PER_I': 11,'POH_I': 12,
 'TRM_I': 13,
 'TIM_B': 14,
 'ANM_B': 15,
 'O': 16,
 'DAT_I': 17,
 'DUR_I': 18,
 'PNT_B': 19,
 'PNT_I': 20,
 'MNY_I': 21,
 'EVT_B': 22,
 'ORG_I': 23,
 'MNY_B': 24,
 'ORG_B': 25,
 'LOC_B': 26,
 'PLT_I': 27,
 'MAT_B': 28,
 'DAT_B': 29,
 'NUM_I': 30,
 'AFW_I': 31,
 'PLT_B': 32,
 'DUR_B': 33,
 'PER_B': 34,
 'LOC_I': 35,
 'NOH_B': 36,
 'TRM_B': 37,
 'NOH_I': 38}
unique_tags={'POH_B', 'CVL_B', 'FLD_B', 'FLD_I', 'EVT_I', 'AFW_B', 'CVL_I', 'NUM_B', 'MAT_I', 'TIM_I', 'ANM_I', 'PER_I', 'POH_I', 'TRM_I', 'TIM_B', 'ANM_B', 'O', 'DAT_I', 'DUR_I', 'PNT_B', 'PNT_I', 'MNY_I', 'EVT_B', 'ORG_I', 'MNY_B', 'ORG_B', 'LOC_B', 'PLT_I', 'MAT_B', 'DAT_B', 'NUM_I', 'AFW_I', 'PLT_B', 'DUR_B', 'PER_B', 'LOC_I', 'NOH_B', 'TRM_B', 'NOH_I'}
id2tag={0: 'POH_B',
 1: 'CVL_B',
 2: 'FLD_B',
 3: 'FLD_I',
 4: 'EVT_I',
 5: 'AFW_B',
 6: 'CVL_I',
 7: 'NUM_B',
 8: 'MAT_I',
 9: 'TIM_I',
 10: 'ANM_I',
 11: 'PER_I',
 12: 'POH_I',
 13: 'TRM_I',
 14: 'TIM_B',
 15: 'ANM_B',
 16: 'O',
 17: 'DAT_I',
 18: 'DUR_I',
 19: 'PNT_B',
 20: 'PNT_I',
 21: 'MNY_I',
 22: 'EVT_B',
 23: 'ORG_I',
 24: 'MNY_B',
 25: 'ORG_B',
 26: 'LOC_B',
 27: 'PLT_I',
 28: 'MAT_B',
 29: 'DAT_B',
 30: 'NUM_I',
 31: 'AFW_I',
 32: 'PLT_B',
 33: 'DUR_B',
 34: 'PER_B',
 35: 'LOC_I',
 36: 'NOH_B',
 37: 'TRM_B',
 38: 'NOH_I'}
pad_token_id = tokenizer.pad_token_id # 0
cls_token_id = tokenizer.cls_token_id # 101
sep_token_id = tokenizer.sep_token_id # 102
pad_token_label_id = tag2id['O']    # tag2id['O']
cls_token_label_id = tag2id['O']
sep_token_label_id = tag2id['O']

model = AutoModelForTokenClassification.from_pretrained('saving_folder', num_labels=len(unique_tags))
model.to(device)

C:\Users\hi\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


ElectraForTokenClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(54343, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Laye

In [3]:
# 기존 토크나이저는 wordPiece tokenizer로 tokenizing 결과를 반환합니다.
# 데이터 단위를 음절 단위로 변경했기 때문에, tokenizer도 음절 tokenizer로 변경

# berttokenizer를 사용하는데 한국어 vocab이 8000개 정도 밖에 없고 그 안의 한국어들의 거의 음절로 존재
# -> 음절 단위 tokenizer를 적용하면 vocab id를 어느 정도 획득할 수 있어 UNK가 별로 없을듯 하다
def ner_tokenizer(sent, max_seq_length):    
    pre_syllable = "_"
    input_ids = [pad_token_id] * (max_seq_length - 1)
    attention_mask = [0] * (max_seq_length - 1)
    token_type_ids = [0] * max_seq_length
    sent = sent[:max_seq_length-2]

    for i, syllable in enumerate(sent):
        if syllable == '_':
            pre_syllable = syllable
        if pre_syllable != "_":
            syllable = '##' + syllable  # 중간 음절에는 모두 prefix를 붙입니다.
            # 우리가 구성한 학습 데이터도 이렇게 구성되었기 때문이라고 함.
            # 이순신은 조선 -> [이, ##순, ##신, ##은, 조, ##선]
        pre_syllable = syllable

        input_ids[i] = (tokenizer.convert_tokens_to_ids(syllable))
        attention_mask[i] = 1
    
    input_ids = [cls_token_id] + input_ids
    input_ids[len(sent)+1] = sep_token_id
    attention_mask = [1] + attention_mask
    attention_mask[len(sent)+1] = 1
    return {"input_ids":input_ids,
            "attention_mask":attention_mask,
            "token_type_ids":token_type_ids}

In [4]:
def ner_inference(text) :

    model.eval()
    text = text.replace(' ', '/')
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', text)

    predictions , true_labels = [], []

    tokenized_sent = ner_tokenizer(text, len(text)+2)
    input_ids = torch.tensor(tokenized_sent['input_ids']).unsqueeze(0).to(device)
    attention_mask = torch.tensor(tokenized_sent['attention_mask']).unsqueeze(0).to(device)
    token_type_ids = torch.tensor(tokenized_sent['token_type_ids']).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)

    logits = outputs['logits']
    logits = logits.detach().cpu().numpy()
    label_ids = token_type_ids.cpu().numpy()

    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    true_labels.append(label_ids)

    pred_tags = [list(tag2id.keys())[p_i] for p in predictions for p_i in p]

    print('{}\t{}'.format("TOKEN", "TAG"))
    print("===========")
    # for token, tag in zip(tokenizer.decode(tokenized_sent['input_ids']), pred_tags):
    #   print("{:^5}\t{:^5}".format(token, tag))
    for i, tag in enumerate(pred_tags):
        print("{:^5}\t{:^5}".format(tokenizer.convert_ids_to_tokens(tokenized_sent['input_ids'][i]), tag))

In [5]:
text = '240517 저녁예약'
ner_inference(text)

TOKEN	TAG
[CLS]	  O  
  2  	NUM_B
 ##4 	NUM_B
 ##0 	NUM_B
 ##5 	NUM_B
 ##1 	NUM_B
 ##7 	NUM_B
 ##저 	NUM_B
 ##녁 	TIM_B
 ##예 	  O  
 ##약 	  O  
[SEP]	  O  


In [6]:
def ner_inference2(text) : 
  
    model.eval()
    text = text.replace(' ', '/')
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', text)

    predictions , true_labels = [], []
    
    tokenized_sent = ner_tokenizer(text, len(text)+2)
    input_ids = torch.tensor(tokenized_sent['input_ids']).unsqueeze(0).to(device)
    attention_mask = torch.tensor(tokenized_sent['attention_mask']).unsqueeze(0).to(device)
    token_type_ids = torch.tensor(tokenized_sent['token_type_ids']).unsqueeze(0).to(device)    
    
    with torch.no_grad():
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)
        
    logits = outputs['logits']
    logits = logits.detach().cpu().numpy()
    label_ids = token_type_ids.cpu().numpy()

    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    true_labels.append(label_ids)

    pred_tags = [list(tag2id.keys())[p_i] for p in predictions for p_i in p]

    #print('{}\t{}'.format("TOKEN", "TAG"))
    #for i, tag in enumerate(pred_tags):
        #print("{:^5}\t{:^5}".format(tokenizer.convert_ids_to_tokens(tokenized_sent['input_ids'][i]), tag))
    
    #token_list = [tokenizer.convert_ids_to_tokens(tokenized_sent['input_ids'][i]) for i in range(len(tokenized_sent['input_ids']))]
    #token_list = [re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', token) for token in token_list]
    #token_tag_pairs = [(token_list[i], pred_tags[i]) for i in range(len(token_list))]
    
    #for pair in token_tag_pairs:
        #print(pair)
    
    token_list = tokenizer.convert_ids_to_tokens(tokenized_sent['input_ids'])
    token_list = [re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', token) for token in token_list]
    token_tag_pairs = []
    current_tag = None
    current_token = ''
    for token, tag in zip(token_list, pred_tags):
        if tag == current_tag:
            current_token += token
        else:
            if current_token:
                token_tag_pairs.append((current_token, current_tag))
            current_token = token
            current_tag = tag
    if current_token:
        token_tag_pairs.append((current_token, current_tag))

    return token_tag_pairs

In [7]:
text = '오늘 밤 8시부터 9시까지 세미나'
ner_inference2(text)

[('CLS', 'O'),
 ('오늘', 'DAT_B'),
 ('밤', 'TIM_B'),
 ('8시부터9시까지', 'TIM_I'),
 ('세미나SEP', 'O')]

## 최종 ner모델 = ner_inference3함수

In [8]:
def ner_inference3(text):
    model.eval()
    text = text.replace(' ', '/')
    text = re.sub('[-=+#/\?:^$.@*\"※&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', text)

    predictions, true_labels = [], []

    tokenized_sent = ner_tokenizer(text, len(text) + 2)
    input_ids = torch.tensor(tokenized_sent['input_ids']).unsqueeze(0).to(device)
    attention_mask = torch.tensor(tokenized_sent['attention_mask']).unsqueeze(0).to(device)
    token_type_ids = torch.tensor(tokenized_sent['token_type_ids']).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)

    logits = outputs['logits']
    logits = logits.detach().cpu().numpy()
    label_ids = token_type_ids.cpu().numpy()

    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    true_labels.append(label_ids)

    pred_tags = [list(tag2id.keys())[p_i] for p in predictions for p_i in p]

    token_list = tokenizer.convert_ids_to_tokens(tokenized_sent['input_ids'])
    token_list = [re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', token) for token in token_list]

    token_tag_pairs = []
    rest= []
    current_tag = None
    current_token = ''
    for token, tag in zip(token_list, pred_tags):
        if tag in ['NUM_B','NUM_I','DUR_B','DUR_I', 'TIM_B','TIM_I', 'DAT_B','DAT_I']:
            token_tag_pairs.append(token)
            tokenized_text = ''.join(token_tag_pairs)
        else:
            rest.append(token)
        
    rest = rest[1:-1]
    rest = ''.join(rest)
            
    return tokenized_text, rest

In [9]:
text = '오늘 밤 8시부터 9시까지 세미나'
ner_inference3(text)

('오늘밤8시부터9시까지', '세미나')

In [10]:
text = '5월14일부터 5월19일까지 부산여행 간다'
ner_inference3(text)

('5월14일부터5월19일까지', '부산여행간다')

In [11]:
text = '20240516부터 20240518까지 강원도 강릉 출장있음 ㅜㅜ'
ner_inference3(text)

('20240516부터20240518까지', '강원도강릉출장있음ㅜㅜ')

In [12]:
ner_inference3('3일뒤에경복궁 관련 발표있음')

('3일뒤에', '경복궁관련발표있음')

### 이제 라벨 11개 나오게 해야지...

In [51]:
def extract_schedule_info(sentence,event):
    if '부터' in sentence: #부터만 쓰고 까지를 쓰지 않는 경우 종료기간 정규표현식에 들어갈 수 있게끔
         sentence = sentence + "까지"
    if ('부터' in sentence) and ('에' in sentence):
        sentence = sentence.replace('에','까지')
    if 'UNK' in sentence: #특수기호로 기간을 나타내는 경우 문자로 대체해서 정규표현식에 들어갈 수 있게끔
        sentence=sentence.replace("UNK", "부터")
        sentence = sentence + "까지"
    days = ['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일']
    weeks = ['저번주', '이번주', '다음주', '다다음주', '다다다음주']

    # 조건 확인
    if any(day in sentence for day in days) and not any(week in sentence for week in weeks):
        sentence = "이번주 " + sentence
        
    
    
    text = re.sub('[-=+,#/\?:^$.@*\"~※&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', sentence)  # 특수문자 제거
    text = text.replace(" ", "")  # 모든 공백 제거
    
    # 시작 기간을 추출하는 정규표현식
    start_time_pattern = r'(?:(그제|어제|오늘|낼|내일|모래|모레|이번주|다음주|다다음주|\d+년|\d{8}|\d{7}|\d{6}|\d{5}|\d{4}))?(?:(월요일|화요일|수요일|목요일|금요일|토요일|일요일|월|화|수|목|금|토|일))?(?:(\d+)월)?(?:(\d+)일)?(?:(오전|오후|새벽|밤|낮|저녁|아침))?(?:(\d+)시)?(?:(반|\d+분))?(?:부터)?' # 시작 기간 추출
    match = re.search(start_time_pattern, text)
    
        
    if match:
        syear = match.group(1) if match.group(1) else datetime.now().year  # 년도를 추출합니다. 년도가 없으면 기본값으로 2024를 사용합니다.
        smonth = match.group(3).strip("월") if match.group(3) else datetime.now().month
        sday = match.group(4).strip("일") if match.group(4) else datetime.now().day
        sdow = match.group(2) 
        
        if match.group(1) and match.group(1).isdigit():
            if len(match.group(1)) == 8:  # yyyyddmm 형식
                syear = int(match.group(1)[:4])
                smonth = int(match.group(1)[4:6])
                sday = int(match.group(1)[6:])
            elif len(match.group(1)) == 6:  # yyddmm 형식
                syear = int(match.group(1)[:2]) + 2000
                smonth = int(match.group(1)[2:4])
                sday = int(match.group(1)[4:])
            elif len(match.group(1)) == 5:  # yydm 형식
                syear = int(match.group(1)[:2]) + 2000
                sday_month = int(match.group(1)[2:])
                smonth = sday_month // 100
                sday = sday_month % 100
            elif len(match.group(1)) == 7:  # yyyydm 형식
                syear = int(match.group(1)[:4])
                sday_month = int(match.group(1)[4:])
                smonth = sday_month // 100
                sday = sday_month % 100
            elif len(match.group(1)) == 4:
                syear = datetime.now().year
                smonth=int(match.group()[:2])
                sday=int(match.group(1)[2:])

        # 유효한 날짜인지 확인
            if not (1 <= smonth <= 12 and 1 <= sday <= 31):
                smonth = None
                sday = None
        
        
        elif match.group(1) == '오늘':
            syear = datetime.now().year
            smonth = datetime.now().month
            sday = datetime.now().day
        elif match.group(1) == '어제':
            date = datetime.now() - timedelta(days=1)
            syear = date.year
            smonth = date.month
            sday = date.day
        elif (match.group(1) == '내일') or (match.group(1) == '낼') :
            date = datetime.now() + timedelta(days=1)
            syear = date.year
            smonth = date.month
            sday = date.day
        elif (match.group(1) == '모레') or (match.group(1) == '모래'):
            date = datetime.now() + timedelta(days=2)
            syear = date.year
            smonth = date.month
            sday = date.day
        elif match.group(1) == '그제':
            date = datetime.now() - timedelta(days=2)
            syear = date.year
            smonth = date.month
            sday = date.day

            
        elif match.group(1) == '다음주':
            if match.group(2):  # 요일이 주어진 경우
                today = datetime.now().date()
                weekday_str = match.group(2)
                weekdays = {
                    '월요일': 0,
                    '화요일': 1,
                    '수요일': 2,
                    '목요일': 3,
                    '금요일': 4,
                    '토요일': 5,
                    '일요일': 6
                }
                target_weekday = weekdays[weekday_str]
                current_weekday = today.weekday()
                days_to_target = target_weekday - current_weekday
                #if days_to_target >=0:  # 이미 해당 요일이 지난 경우
                days_to_target += 7
                date = datetime.now() + timedelta(days=days_to_target)
                syear = date.year
                smonth = date.month
                sday = date.day
                
                    
            else:  # 요일이 주어지지 않은 경우
                date = datetime.now() + timedelta(days=7)
                syear = date.year
                smonth = date.month
                sday = date.day

            
        elif match.group(1) == '다다음주':
             if match.group(2):  # 요일이 주어진 경우
                today = datetime.now().date()
                weekday_str = match.group(2)
                weekdays = {
                    '월요일': 0,
                    '화요일': 1,
                    '수요일': 2,
                    '목요일': 3,
                    '금요일': 4,
                    '토요일': 5,
                    '일요일': 6
                }
                target_weekday = weekdays[weekday_str]
                current_weekday = today.weekday()
                days_to_target = target_weekday - current_weekday
                #if days_to_target >=0:  # 이미 해당 요일이 지난 경우
                days_to_target += 14
                date = datetime.now() + timedelta(days=days_to_target)
                syear = date.year
                smonth = date.month
                sday = date.day
                
            
             else:
                date = datetime.now() + timedelta(days=14)
                syear = date.year
                smonth = date.month
                sday = date.day
            
            
        elif match.group(1) == '이번주':
            today = datetime.now().date()
            weekday_str = match.group(2)
            weekdays = {
                '월요일': 0,
                '화요일': 1,
                '수요일': 2,
                '목요일': 3,
                '금요일': 4,
                '토요일': 5,
                '일요일': 6
            }
            target_weekday = weekdays[weekday_str]
            current_weekday = today.weekday()
            days_to_target = (target_weekday - current_weekday) % 7
            date = datetime.now() + timedelta(days=days_to_target)
            syear = date.year
            smonth = date.month
            sday = date.day
        elif match.group(1) and '년' in match.group(1):
            syear = int(match.group(1).strip("년"))
        else:
            syear = int(datetime.now().year)
    
            
        stime_period = match.group(5)
        shour = int(match.group(6).strip("시")) if match.group(6) else 0
        
        
        if (stime_period == '오후' or stime_period == '저녁' or stime_period == '밤' or stime_period == '낮') and int(shour) < 12:
            shour = int(shour) + 12
        if match.group(7) and '반' in match.group(7):
            sminute = 30
        elif match.group(7):
            sminute = int(match.group(7).strip("분"))
        else:
            sminute = 0
        
        # 종료 기간을 추출하는 정규표현식
        end_time_pattern = r'(?:(어제|오늘|내일|모레|다음주|다다음주|\d+년|\d{8}|\d{7}|\d{6}|\d{5}|\d{4}))?(?:(월|화|수|목|금|토|일|월요일|화요일|수요일|목요일|금요일|토요일|일요일))?(?:(\d+)월)?(?:(\d+)일)?(?:(오전|오후|새벽|밤|낮|저녁|아침))?(?:(\d+)시)?(?:(반|\d+분))?(까지)' # 종료 기간 추출
        end_match = re.search(end_time_pattern, text)
        if end_match:
            fyear = end_match.group(1) if end_match.group(1) and end_match.group(1).isdigit() else syear  # 종료 년도를 추출합니다.
            fmonth = end_match.group(3).strip("월") if end_match.group(3) else smonth
            fday = end_match.group(4).strip("일") if end_match.group(4) else sday
            ftime_period = end_match.group(5)
            fhour = int(end_match.group(6).strip("시")) if end_match.group(6) else 0
            
            if end_match.group(1) and end_match.group(1).isdigit():
                if len(end_match.group(1)) == 8:  # yyyyddmm 형식
                    fyear = int(end_match.group(1)[:4])
                    fmonth = int(end_match.group(1)[4:6])
                    fday = int(end_match.group(1)[6:])
                elif len(end_match.group(1)) == 6:  # yyddmm 형식
                    fyear = int(end_match.group(1)[:2]) + 2000
                    fmonth = int(end_match.group(1)[2:4])
                    fday = int(end_match.group(1)[4:])
                elif len(end_match.group(1)) == 5:  # yydm 형식
                    fyear = int(end_match.group(1)[:2]) + 2000
                    fday_month = int(end_match.group(1)[2:])
                    fmonth = fday_month // 100
                    fday = fday_month % 100
                elif len(end_match.group(1)) == 7:  # yyyydm 형식
                    fyear = int(end_match.group(1)[:4])
                    fday_month = int(end_match.group(1)[4:])
                    fmonth = fday_month // 100
                    fday = fday_month % 100
                elif len(end_match.group(1)) == 4:
                    fyear = datetime.now().year
                    fmonth=int(end_match.group()[:2])
                    fday=int(end_match.group(1)[2:])

            # 유효한 날짜인지 확인
                if not (1 <= fmonth <= 12 and 1 <= fday <= 31):
                    fmonth = None
                    fday = None
            
            
            elif end_match.group(1) == '다음주':
                if end_match.group(2):  # 요일이 주어진 경우
                    today = datetime.now().date()
                    weekday_str2 = end_match.group(2)
                    weekdays = {
                    '월요일': 0,
                    '화요일': 1,
                    '수요일': 2,
                    '목요일': 3,
                    '금요일': 4,
                    '토요일': 5,
                    '일요일': 6
                    }
                    target_weekday2 = weekdays[weekday_str2]
                    current_weekday2 = today.weekday()
                    days_to_target2 = target_weekday2 - current_weekday2
                    #if days_to_target >=0:  # 이미 해당 요일이 지난 경우
                    days_to_target2 += 7
                    date = datetime.now() + timedelta(days=days_to_target2)
                    fyear = date.year
                    fmonth = date.month
                    fday = date.day
                    
                else:  # 요일이 주어지지 않은 경우
                    date = datetime.now() + timedelta(days=7)
                    fyear = date.year
                    fmonth = date.month
                    fday = date.day

            elif end_match.group(1) == '다다음주':
                if end_match.group(2):  # 요일이 주어진 경우
                    today = datetime.now().date()
                    weekday_str2 = end_match.group(2)
                    weekdays = {
                    '월요일': 0,
                    '화요일': 1,
                    '수요일': 2,
                    '목요일': 3,
                    '금요일': 4,
                    '토요일': 5,
                    '일요일': 6
                    }
                    target_weekday2 = weekdays[weekday_str2]
                    current_weekday2 = today.weekday()
                    days_to_target2 = target_weekday2 - current_weekday2
                    #if days_to_target >=0:  # 이미 해당 요일이 지난 경우
                    days_to_target2 += 14
                    date = datetime.now() + timedelta(days=days_to_target2)
                    fyear = date.year
                    fmonth = date.month
                    fday = date.day
                
                else:
                    date = datetime.now() + timedelta(days=14)
                    fyear = date.year
                    fmonth = date.month
                    fday = date.day
                
            elif end_match.group(1) == '이번주':
                today = datetime.now().date()
                weekday_str2 = end_match.group(2)
                weekdays = {
                 '월요일': 0,
                '화요일': 1,
                '수요일': 2,
                '목요일': 3,
                '금요일': 4,
                '토요일': 5,
                '일요일': 6
                }
            
                target_weekday2 = weekdays[weekday_str2]
                current_weekday2 = today.weekday()
                days_to_target2 = (target_weekday2 - current_weekday2) % 7
                date = datetime.now() + timedelta(days=days_to_target2)
                fyear = date.year
                fmonth = date.month
                fday = date.day
            elif end_match.group(1) and '년' in end_match.group(1):
                fyear = int(end_match.group(1).strip("년"))
            else:
                fyear = datetime.now().year
                 
            if (ftime_period == '오후' or ftime_period == '저녁' or ftime_period == '밤' or ftime_period == '낮') and int(fhour) < 12:
                fhour = int(fhour) + 12
                
            
            if end_match.group(7) and '반' in end_match.group(7):
                fminute = 30
            elif end_match.group(7):
                fminute = int(end_match.group(7).strip("분"))
            else:
                fminute = 0

                
        
        else:
            fyear, fmonth, fday, ftime_period, fhour, fminute = syear, smonth, sday, stime_period, shour, sminute

        return syear, smonth, sday, stime_period, shour, sminute, fyear, fmonth, fday,ftime_period,  fhour, fminute,event
    else:
        return None, None, None, None, None, None, None,None, None, None, None, None,None

In [54]:
answer=ner_inference3('내일 10시 반에 컴퓨터 써야돼')
extract_schedule_info(answer[0],answer[1])

(2024, 6, 25, None, 10, 30, 2024, 6, 25, None, 10, 30, '컴퓨터써야돼')

### 정밀도 (Precision): 모델이 양성으로 예측한 샘플 중에서 실제로 양성인 샘플의 비율을 나타냅니다. 예측한 양성 중에서 얼마나 정확하게 예측했는지를 측정하는 지표입니다. 정밀도는 FP(False Positive)를 줄이는 것이 목표일 때 중요합니다.

### 재현율 (Recall): 실제 양성인 샘플 중에서 모델이 양성으로 예측한 샘플의 비율을 나타냅니다. 실제 양성 중에서 얼마나 많은 샘플을 찾아내는지를 측정하는 지표입니다. 재현율은 FN(False Negative)를 줄이는 것이 목표일 때 중요합니다.

### F1 점수 (F1 Score): 정밀도와 재현율의 조화 평균으로, 이 둘의 균형을 평가하는 지표입니다. 정밀도와 재현율 사이의 균형을 나타내며, 이 둘 중 하나만 고려하는 것보다 더 종합적인 성능 평가를 제공합니다.

### 혼동 행렬 (Confusion Matrix): 이는 모델이 어떻게 분류했는지를 보여주는 표입니다. 주로 이진 또는 다중 분류 모델의 결과를 시각화할 때 사용됩니다.